In [446]:
import pandas as pd
import natsort as ns

### Setup ###

In [440]:
def setup(file_name: str):
    df = pd.read_csv(file_name)
    return df

def save_to_file(content: str, file_name:str, t: str):
    with open(file_name, t) as file:
        file.write(content)
        file.close()

### Continuity Sandbox ###

In [441]:
def to_output(row):
    pin = "-" + str(row["PIN LEFT"]) + "\n" if row["PIN LEFT"] != "" else "\n"
    return "\nX-" + str(row["FROM"]) + pin

def to_input_continuity(group):
    if len(group) > 1:
        sorted_cont_pts = pd.Series(ns.natsorted(group.apply(to_multiple_continuity, axis=1)))
        sorted_cont_pts.iloc[-1] = sorted_cont_pts.iloc[-1].replace("CV-", "C-")
        return pd.Series(sorted_cont_pts.agg("sum"))
    return group.apply(to_single_continuity, axis=1).reset_index(drop=True)
        
def to_single_continuity(row):
    pin =  "-" + str(row["PIN RIGHT"]) + "\n" if row["PIN RIGHT"] != "" else "\n"
    return "C-" + str(row["TO"]) + pin

def to_multiple_continuity(row):
    pin = "-" + str(row["PIN RIGHT"]) + "\n" if row["PIN RIGHT"] != "" else "\n"
    return "CV-" + str(row["TO"]) + pin

def convert_continuity(df: pd.DataFrame):
    cont_df = df.copy()
    cont_df["output"] = cont_df.apply(to_output, axis=1)
    cont_df = cont_df.groupby("output").apply(to_input_continuity, include_groups=False).reset_index().rename(columns={0:"input"})
    cont_df["continuity"] = pd.Series(ns.natsorted(cont_df.apply("sum", axis=1)))
    return cont_df
    
def continuity_to_test(df: pd.DataFrame, block_name: str, params: str):
    cont_df = convert_continuity(df)
    header = f"BTB, {block_name}\n" + params + "\n"
    footer = f"\nETB, {block_name}\n"
    return header + cont_df["continuity"].aggregate("sum", axis=0) + footer

In [443]:
df = setup("spreadsheets/wire_list.csv")
pars = "C, 0.1A, <2R, 0.01s"
cont_test = continuity_to_test(df, "CONTINUITY_TESTS", pars)
save_to_file(cont_test, "continuity_block_testing.txt", "w")

### Isolation Sandbox ###

In [134]:
def to_input_isolation(row):
    return "\nT-" + str(row["Connector"]) + "-" + str(row["Pin"]) + "\n"

def convert_isolation(df: pd.DataFrame):
    sorted_con_pin = ns.natsorted(df.apply(to_input_isolation, axis=1))
    column_name = "isolation"
    return pd.DataFrame({column_name: sorted_con_pin})

def isolation_to_test(df: pd.DataFrame, block_name: str, params: str):
    iso_df = convert_isolation(df)
    header = f"BTB, {block_name}\n" + params + "\n"
    footer = f"ETB, {block_name}\n"
    return header + iso_df["isolation"].aggregate("sum", axis=0) + footer

In [ ]:
df = setup("unused_pins.csv")
pars = "T, 500V, >2M, 0.15T, 0.3s"
iso_test = isolation_to_test(df, "ISOLATION_TESTS", pars)
save_to_file(iso_test, "isolation_block.txt", "w")

### HIPOT Sandbox ###

In [124]:
def to_input_hipot(row):
    return "\nD-" + str(row["Connector"]) + "-" + str(row["Pin"]) + "\n"

def convert_hipot(df: pd.DataFrame):
    left = df[['FROM', 'PIN LEFT']].rename(columns={'FROM': "Connector", "PIN LEFT": "Pin"})
    right = df[['TO', 'PIN RIGHT']].rename(columns={'TO': "Connector", "PIN RIGHT": "Pin"})
    connectors = pd.concat([left, right], ignore_index=True).drop_duplicates(subset=['Connector', 'Pin'])
    column_name = "hipot"
    sorted_con_pin = ns.natsorted(connectors.apply(to_input_hipot, axis=1))
    return pd.DataFrame({column_name: sorted_con_pin})

def hipot_to_test(df: pd.DataFrame, block_name: str, params: str):
    hipot_df = convert_hipot(df)
    header = f"BTB, {block_name}\n" + params + "\n"
    footer = f"ETB, {block_name}\n"
    return header + hipot_df["hipot"].aggregate("sum", axis=0) + footer

In [125]:
df = setup("wire_list.csv")
pars = "D, HIP, 500V, 0.2 MA, 0.2T, 60.0s"
hipot_test = hipot_to_test(df, "HIPOT_TESTS", pars)
save_to_file(hipot_test, "hipot_block.txt", "w")

## Ground Sandbox ##

In [257]:
def setup():
    adapters = pd.read_csv("spreadsheets/mappings.csv")
    grd_conn = pd.read_csv("spreadsheets/fts_ground.csv")
    return adapters, grd_conn

In [265]:
def convert(name, exceptions: list[str]):
    if not name in exceptions:
        if name[0] == "J":
            name = "P" + name[1:]
        elif name[0] == "P":
            name = "J" + name[1:]
    return name

def convert_adapters(df, exceptions: list[str]):
    df["AdapterA"] = df["ConnectorA"].apply(lambda x: convert(x, exceptions=exceptions))
    df["AdapterB"] = df["ConnectorB"].apply(lambda x: convert(x, exceptions=exceptions))
    return df

def map_ground_adp(grd, connectors: pd.DataFrame):
    adapters = connectors.drop(columns=["ConnectorA", "ConnectorB"])
    grd_con = pd.merge(left=grd, right=adapters, left_on="Adapter", right_on="AdapterA").drop(columns=["Adapter"]).rename(columns={"Ground": "GrdA"})
    grd_con = pd.merge(left=grd, right=grd_con, left_on="Adapter", right_on="AdapterB").drop(columns=["Adapter"]).rename(columns={"Ground": "GrdB"})
    return grd_con

def format_ground(df: pd.DataFrame):
    df["PIN LEFT"] = pd.Series([""] * len(df), name="PIN LEFT")
    df["PIN RIGHT"] = pd.Series([""] * len(df), name="PIN RIGHT")
    df.drop(columns=["AdapterA", "AdapterB"], inplace=True)
    df.rename(columns={"GrdA": "FROM", "GrdB": "TO"}, inplace=True)
    return df

In [400]:
adp_df, grd_df = setup()

In [405]:
grd_df = convert_adapters(grd_df, exceptions=["J699"])
grd_map = map_ground_adp(adp_df, grd_df)

In [406]:
formatted = format_ground(grd_map)

In [438]:
cont_test_grd = continuity_to_test(formatted, "CONTINUITY_TESTS", pars)
save_to_file(cont_test_grd, "results/grd_cont_block_test.txt", "w")

CONT DF
      output            input
0  \nX-J10\n          C-J14\n
1  \nX-J12\n  CV-J11\nC-J20\n
2  \nX-J15\n           C-J1\n
3  \nX-J16\n  CV-J11\nC-J20\n
4  \nX-J17\n           C-J4\n
5   \nX-J2\n  CV-J11\nC-J20\n
6   \nX-J3\n           C-J1\n
7   \nX-J5\n           C-J4\n
8   \nX-J6\n  CV-J12\nC-J16\n


## Simple Ground ##

In [444]:
grd_conn = pd.read_csv("spreadsheets/ground_connections.csv")
grd_conn["PIN LEFT"] = pd.Series([""] * len(grd_conn), name="PIN LEFT")
grd_conn["PIN RIGHT"] = pd.Series([""] * len(grd_conn), name="PIN RIGHT")
grd_conn.rename(columns={"grdA": "FROM", "grdB": "TO"}, inplace=True)

In [445]:
cont_test_grd = continuity_to_test(grd_conn, "CONTINUITY_TESTS", pars)
save_to_file(cont_test_grd, "results/grd_cont_block_test.txt", "w")

In [492]:
df = pd.DataFrame({"A": [1, 2, 1, 8], "B": [7, 8, 7, 8], "C": [4, 5, 4, 8] })

In [493]:
df

,A,B,C
0,1,7,4
1,2,8,5
2,1,7,4
3,8,8,8


In [494]:
a = df.apply(lambda row: tuple(sorted(row)), axis=1).drop_duplicates().index.to_list()

In [495]:
filtered = df.iloc[a]

In [496]:
filtered.reset_index()

,index,A,B,C
0,0,1,7,4
1,1,2,8,5
2,3,8,8,8


In [502]:
row = [2, 8, 5]

In [504]:
new_row = pd.DataFrame({df.columns.to_list()[i]:[row[i]] for i in range(len(df.columns))})
ok = pd.concat([df, new_row])
indices = ok.apply(lambda row: tuple(sorted(row)), axis=1).drop_duplicates().index.to_list()
ok.iloc[indices].reset_index().drop(columns=["index"])

,A,B,C
0,1,7,4
1,2,8,5
2,8,8,8


In [4]:
a = "hi si"

In [5]:
b = "fo fi"

In [6]:
c = [a, b]

In [9]:
ok = []
for p in c:

    ok += p.split(" ")

In [10]:
ok

['hi', 'si', 'fo', 'fi']